# MLP training on MNIST

1.   Testing on all 10000 images,because faster. Problem?
2.   Listeneintrag



In [ ]:
import numpy as np
import jax.numpy as jnp
from jax.scipy.special import logsumexp
import jax
import pickle
from jax import jit, vmap, pmap, grad, value_and_grad
import random
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from jax.example_libraries import stax, optimizers
import torchvision
import torch
from sklearn.neighbors import NearestNeighbors
import torch.utils.data as data_utils
from jax.flatten_util import ravel_pytree
import os
import time
import shutil
import _pickle as cPickle
import time
from jax.example_libraries import stax
from jax.example_libraries.stax import Dense, Relu, LogSoftmax
from sklearn.model_selection import train_test_split
from jax import random 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Variables
Most variables overriden in main code part!!!

In [ ]:
#Needs Cleaning

'''Set your file directorys'''
file_name="JAX_MNist_parallel1"
googledrive_path="/content/drive/MyDrive/Colab Notebooks/Jax_MNist/"
local_path="C:/Users/Flo/Documents/Uni/Masterarbeit/Hanabi/Mnist handwritten digits/"





'''number of training epochs for Network2'''
n_training_epochs = 3 #number of training epochs for every NN2.  
n_offsp_epoch = 10 #number of training and testing runs combined, to get an average for the performance of the Convu net.
n_testing_epochs = 3 # number of testing runs, per n_offsp_epoch

'''parameter Network2'''
n_samples = 150  #Number of training samples for NN2, distribution of data for 600: [68, 59, 66, 67, 47, 46, 65, 71, 53, 58])
n_test=1000 #Number of test samples for NN2. Needs to be multiples of batch_size_test=500 )
batch_size = 50 # for precise n_samples number must be: n_samples%batch_size_train=0

learning_rate = 0.1
momentum = 0.5
log_interval = 10



'''Standard deviation for gaussian noise in Network1'''
'''!!! Important hyperparameter, >=1 gives bad results'''
std_modifier=0.05



'''number of offsprings per metaepoch'''
n_offsprings=100
'''number of metaopochs'''
n_metaepochs=10


NNin1=2500 #dependent on Convu
NNout1=10



'''Convunet'''
Convu1_in=1
Convu2_in=12
Convu3_in=24
seed_convu=0

n_samples=150 #number of training samples
batch_size = 50
n_test=1000
n_training_epochs=3
print_distribution_data=False
std_modifier=0.05


Convu1_in=32
Convu2_in=16
Convu3_in=4



use_sigma_decay=True
sigma_start=1.0
sigma_goal=0.1

'''logging to screen variables'''
print_offsprings=True
print_distribution_data=False
use_sigma_decay=True #otherwise using constant sigma from config tab
sigma_start=1 
sigma_goal=0.05 #sigma goal after n_metaepochs
        
'''choose either method, softmax or elitist=keep only best offspring'''
use_softmax=True
temperature=0.05
use_elitist=False
use_winnerlist=False

n_metaepochs=30 #overwriting variable from config tab, delete later
n_offsprings=10 #overwriting variable from config tab, delete later

'''number of training epochs for Network2'''
n_offsp_epoch = 2
n_testing_epochs = 5
n_metaepochs=10

## **Funktions**

In [ ]:
def pathandstuff():

    global save_txt
    global base_path
    global save_path

    if os.path.exists(local_path):
        '''Save running code file to log folder'''
        #nb_full_path = os.path.join(os.getcwd(), nb_name) #path of current notebook
        #shutil.copy2(nb_full_path, save_path) #save running code file to log folder
        print("on local")
        base_path=local_path
    elif os.path.exists(googledrive_path):
        print("on google")
        base_path=googledrive_path
    else:
        raise ValueError('Please specify save path or connect to Google Drive')
        
    logs_path=base_path+"Logs/"
    '''Set logging and temp paths'''
    timestamp=time.strftime("%d.%m.%Y_%H.%M")
    foldername=timestamp
    save_path="/content/drive/MyDrive/Colab Notebooks/Jax_MNist/Logs/19.04.2022_2parallel/"
    #save_path=os.path.join(logs_path,foldername,)
    #save_path=save_path+"/"
    print("Log path:",save_path)
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    save_txt = os.path.join(save_path, 'Log_Jax_MNist_Nr1.txt')


def logg_script(file_name, save_path):
  source=f"/content/drive/MyDrive/Colab Notebooks/{file_name}"
  destination=save_path+f"{file_name}.ipynb"
  shutil.copy2(source, destination)

'''logging to txt and print'''
def logg (string_, array=None):
  if array is None:

    file1 = open(save_txt,"a")
    file1.write(string_)
    file1.write("\n")
    file1.close()
    print(string_)
  if array is not None:

    file1 = open(save_txt,"a")
    file1.write(string_)
    file1.write(str(array))
    file1.write("\n")
    file1.close()
    print(string_, array)

In [ ]:
train_dataset = MNIST(root='train_mnist', train=True, download=True,transform=torchvision.transforms.Compose([
                                            torchvision.transforms.ToTensor(),
                                            torchvision.transforms.Normalize((0.1307,), (0.3081,))]))

test_dataset = MNIST(root='test_mnist', train=False, download=True,transform=torchvision.transforms.Compose([
                                            torchvision.transforms.ToTensor(),
                                            torchvision.transforms.Normalize((0.1307,), (0.3081,))]))

x = np.concatenate((train_dataset.data,test_dataset.data))
y= np.concatenate((train_dataset.targets,test_dataset.targets))

x = jnp.array(x,dtype="float32").reshape(len(x), -1)
y = jnp.array(y)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting train_mnist/MNIST/raw/train-images-idx3-ubyte.gz to train_mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting train_mnist/MNIST/raw/train-labels-idx1-ubyte.gz to train_mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

...

Extracting test_mnist/MNIST/raw/train-labels-idx1-ubyte.gz to test_mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting test_mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to test_mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting test_mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to test_mnist/MNIST/raw



In [ ]:
def init_MLP(layer_widths, parent_key, scale=0.01):

    params = []
    keys = jax.random.split(parent_key, num=len(layer_widths)-1)

    for in_width, out_width, key in zip(layer_widths[:-1], layer_widths[1:], keys):
        weight_key, bias_key = jax.random.split(key)
        params.append([
                       scale*jax.random.normal(weight_key, shape=(out_width, in_width)),
                       scale*jax.random.normal(bias_key, shape=(out_width,))
                       ]
        )
    return params



In [ ]:
@jit
def MLP_predict(params, x):

    hidden_layers = params[:-1]
    activation = x

    for w, b in hidden_layers:
        activation = jax.nn.relu(jnp.dot(w, activation) + b)

    w_last, b_last = params[-1]
    logits = jnp.dot(w_last, activation) + b_last

    return logits - logsumexp(logits)

jit_MLP_predict=jit(MLP_predict)

@jit
def batched_MLP_predict(params,x):
  return vmap(jit_MLP_predict, (None, 0))(params,x)
  
jit_batched_MLP_predict=jit(batched_MLP_predict)


In [ ]:
Convu1_in=16
Convu2_in=24
Convu3_in=1

conv_init, conv_apply = stax.serial(
    stax.Conv(Convu1_in,(3,3), padding="SAME"),
    stax.BatchNorm(),
    stax.Relu,
    stax.MaxPool((2,2)),
    stax.Conv(Convu2_in, (3,3), padding="SAME"),
    stax.BatchNorm(),
    stax.Relu,
    stax.MaxPool((2,2)),
    stax.Conv(Convu3_in, (3,3), padding="SAME"),
    stax.Relu,
    stax.MaxPool((2,2))
)


In [ ]:
'''After changing Convu structure test if convu out and NN in matches, set NNin1=25*25*4 to corresponding shape in error (5, 25, 25, 4) '''
NNin1=625
rng=jax.random.PRNGKey(1)

father_weights = conv_init(rng, (batch_size,28,28,1))
father_weights = father_weights[1]


x_train=x[random.randint(rng, (n_offsp_epoch*n_samples,), 0, 60000, dtype='uint8')]
testaffe=x_train[0:5]
imgs = conv_apply(father_weights, testaffe.reshape(-1,28,28,1))

MLP_params = init_MLP([NNin1, 10], rng)

pred_classes = jnp.argmax(jit_batched_MLP_predict(MLP_params, imgs.reshape(-1,NNin1)), axis=1)

In [ ]:
@jit
def loss_fn(params, imgs, gt_lbls):
  
    predictions = jit_batched_MLP_predict(params, imgs)
    #print("predictions",predictions.shape)
    return -jnp.mean(predictions * gt_lbls)
    
jit_loss_fn=jit(loss_fn)

@jit
def update(params, imgs, gt_lbls, lr=0.01):
    loss, grads = value_and_grad(loss_fn)(params, imgs, gt_lbls)

    return loss, jax.tree_multimap(lambda p, g: p - lr*g, params, grads)

jit_update=jit(update)

@jit
def accuracy(conv_weights,MLP_params, dataset_imgs, dataset_lbls):

    imgs = conv_apply(conv_weights, dataset_imgs.reshape(-1,28,28,1))
    pred_classes = jnp.argmax(jit_batched_MLP_predict(MLP_params, imgs.reshape(-1,NNin1)), axis=1)

    return jnp.mean(dataset_lbls == pred_classes)
    
jit_accuracy=jit(accuracy)


In [ ]:
'''For loop is neccesary to do batch training. Every update iteration needs to run with updated MPL params'''
@jit
def train(conv_weights, imgs, lbls,MLP_params ):
  for n in range(n_training_epochs):  
    for i in range(jnp.shape(imgs)[0]):

      gt_labels = jax.nn.one_hot(lbls[i], 10)
      img_conv = conv_apply(conv_weights, imgs[i].reshape(-1,28,28,1))
      loss, MLP_params = jit_update(MLP_params, img_conv.reshape(-1,NNin1), gt_labels)

  return MLP_params
  
jit_train=jit(train)  

In [ ]:
'''
Running for every offspring n_offsp_epoch loops to get stable acc results. 
Every loop is trained with n_samples/batch_size * batch size training epochs.
Everything put in jit and vmap to speed up

Input  
(10, 6, 25, 28, 28, 1) x_train
(10, 6, 25) y_train
(10, 1000, 28, 28, 1) test_img_off
(10, 1000) test_lbl_off
          
(n_offsp_epoch, n_samples/batch_size, batch size, 28, 28, 1)
(n_offsp_epoch, n_samples/batch_size, batch size)
(n_offsp_epoch, n_test, 28, 28, 1)
(n_offsp_epoch, n_test)'''

@jit
def bootstrapp_offspring_MLP(key,conv_weights, batch_affe, labelaffe,test_images,test_lbls):
  
  
  MLP_params = init_MLP([NNin1, NNout1], key)
  MLP_params_trained=jit_train(conv_weights, batch_affe, labelaffe,MLP_params )
 
  
  result=jit_accuracy(conv_weights,MLP_params_trained,test_images,test_lbls)
  return (result)

jit_bootstrapp_offspring_MLP=jit(bootstrapp_offspring_MLP)  

@jit
def vmap_bootstrapp_offspring_MLP(key, conv_weights, batch_affe, labelaffe,test_images,test_lbls):
  return vmap(jit_bootstrapp_offspring_MLP, ( None,None, 0,0,0,0))(key, conv_weights, batch_affe, labelaffe,test_images,test_lbls)
  
jit_vmap_bootstrapp_offspring_MLP=jit(vmap_bootstrapp_offspring_MLP)


In [ ]:
'''creating offsprings Approach 2'''
def create_offsprings(n_offspr, fath_weights,std_modifier,seed):
  np.random.seed(seed)
  statedic_list=[]
  for i in range(0,n_offspr):
    dicta = [()] * len(fath_weights)
    for idx,w in enumerate(fath_weights):
        if w:
            w, b = w
            #print("Weights : {}, Biases : {}".format(w.shape, b.shape))
      
            '''if weight layer only contains 0 and 1, only copy original weight layer, dont add random noise. Purpose of these 0 and 1 layers unclear'''
            if any(w[0].shape==t for t in [(Convu1_in,) ,(Convu2_in,), (Convu3_in,)]):
              x_w=w
              x_b=b
            else:
              seed=np.random.randint(0,100000)
              key = random.PRNGKey(seed)
              x_w = w+random.normal(key,shape=w.shape)*std_modifier #tested, random.normal adding different random noise value to every single weight
              x_b = b+random.normal(key,shape=b.shape)*std_modifier
            dicta[idx]=(x_w,x_b)
    
    statedic_list.append(dicta)
  return statedic_list

In [ ]:
'''softmax for offspring list for approach 2
    checked 11.04 working correctly'''
def softmax_offlist(off_list,acc_list,temp):
  softmax_list=softmax_result(acc_list,temp)
  for i in range(len(off_list)):
    if i==0:
      top_dog=jax.tree_map(lambda x: x*softmax_list[i], off_list[i])
    else:
      general_dog = jax.tree_map(lambda x: x*softmax_list[i], off_list[i])
      top_dog=jax.tree_map(lambda x,y: x+y, top_dog,general_dog)
  return top_dog

In [ ]:
'''Creates softmax/temp list out of accuracy list [0.2,0.3,....,0.8]'''
def softmax_result(results,temp: float):
    x = [z/temp for z in results]
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def sigma_decay(start, end, n_iter):
  return(end/start)**(1/n_iter)

In [ ]:
def KNN_weight_update(result_list_metaepoch, offspring_list):

  '''acc_list=[x[0] for x in result_list_metaepoch]
  max_value=np.max(acc_list)
  top_twenty=[x[0] for x in result_list_metaepoch if x[0] > max_value*(1-KNN_quantil)]
  index_tw=[acc_list.index(x) for x in top_twenty]'''

  acc_list=[x[0] for x in result_list_metaepoch]
  acc_list.sort(reverse=True)
  top_n=acc_list[:KNN_top_n]
  index_tw=[acc_list.index(x) for x in acc_list[:KNN_top_n]]


  '''Get weights of top twenty'''
  tw_weight_list=[offspring_list[i] for i in index_tw]
  flat_tw_weight_list=[]
  for weight in tw_weight_list:
    affe=jax.flatten_util.ravel_pytree(weight)
    flat_tw_weight_list.append(np.array(affe[0]))
  '''given condition would throw an error, return original list in this case'''
  if len(flat_tw_weight_list) < KNN_n_neighbors:
    KNN_use=False
    return KNN_use,offspring_list
  else:
    knn = NearestNeighbors(n_neighbors=KNN_n_neighbors)
    knn.fit(flat_tw_weight_list)
    distance_mat, neighbours_mat = knn.kneighbors(flat_tw_weight_list)
    offspring_list2=[]
    for liste in neighbours_mat:
      weight_list=[tw_weight_list[i] for i in liste]
      acc_list=[top_n[i] for i in liste]
      new_offspring=softmax_offlist(weight_list,acc_list,temp)
      offspring_list2.append(new_offspring)
      KNN_use=True
    return KNN_use,offspring_list2

In [ ]:
'''creating offsprings Approach 2, filling treeleaf of 0 and 1 with gaussian noise, doesnt seem to be a problem, ex in offspring_list[0][5]'''

def random_split_like_tree(rng_key, target=None, treedef=None):
    if treedef is None:
        treedef = jax.tree_structure(target)
    keys = jax.random.split(rng_key, treedef.num_leaves)
    return jax.tree_unflatten(treedef, keys)


def tree_random_normal_like(rng_key, target,std_modifier):
    keys_tree = random_split_like_tree(rng_key, target)
    return jax.tree_multimap(
        lambda l, k: jax.random.normal(k, l.shape, l.dtype)*std_modifier,
        target,
        keys_tree,
    )

def jax_create_offsprings(key,n_offspr,  fath_weights,std_modifier):
  statedic_list=[]
  for i in range(0,n_offspr):
    rng=jax.random.PRNGKey(key+i)
    random_value_tree=tree_random_normal_like(rng,fath_weights,std_modifier)
    son=jax.tree_map(lambda x,y: x+y, fath_weights,random_value_tree)
    statedic_list.append(son)

  return statedic_list

In [ ]:
'''Initialize Variables'''
import _pickle as cPickle

common_weight_path="/content/drive/MyDrive/Colab Notebooks/Jax_MNist/Logs/19.04.2022_2parallel/"

n_training_epochs=30 #= how many times, is the MLP trained with the same data. Reduces dependance on the initialization of MLP weights

n_samples = 100 #n of training independent training samples for MLP, samples are stratified
batch_size = 25
n_test=1250
n_offsp_epoch=40 #Bootstrapping, delivers more stable results for every offspring

'''keys'''
starting_key=48 #define starting point
MLP_key=685 #seed 
numpy_seed=47 #in create offsprings

use_sigma_decay=True #otherwise using constant sigma from config tab
n_decay_epochs=int(n_metaepochs/1)   # over how many metaepochs sigma is decayed
sigma_start=0.02
sigma_goal=0.000001 #sigma goal after n_metaepochs

'''KNN weight update, disable sigma_decay, start with small sigma'''
use_KNN=False
KNN_n_neighbors=10
KNN_quantil=0.10
KNN_top_n=10
n_KNN_subsprings=50 #number offsprings of every KNN update

use_focus=False
use_Softmax=True
use_pickle=False #load weights
use_best_weights=False
pickle_path="/content/drive/MyDrive/Colab Notebooks/Jax_MNist/Logs/12.04.2022_15.59/best_weight_0.7567.pkl"
use_father=True
std_modifier=0.02
temp=0.05 #weight for softmax


use_offspring_decay=True
offspring_start=400
offspring_goal=50
n_metaepochs=10000
n_offsprings=400

In [ ]:
from math import e
#main code
'''Start Logging'''
pathandstuff()
logg_script(file_name, save_path)
f_idx=0
common_start_acc=0

rng_MLP=jax.random.PRNGKey(MLP_key)
results_meta=[]
best_performer=[0.0,0.0]
father_key=jax.random.PRNGKey(starting_key)
best_weights=conv_init(father_key, (batch_size,28,28,1))[1]

for meta in range (n_metaepochs):

    common_weights_loaded=False
    '''Check for better weight'''
    best_weights_list=os.listdir(common_weight_path)
    if best_weights_list is not None:
      for weights in best_weights_list[::-1]:
        if "best_weight" in weights:
          highest_acc=float(weights.split("best_weight_")[1].split(".pkl")[0])
          if highest_acc > common_start_acc:
            with open(f"/content/drive/MyDrive/Colab Notebooks/Jax_MNist/Logs/19.04.2022_2parallel/best_weight_{highest_acc:.4f}.pkl", "rb") as input_file:
                father_weights = cPickle.load(input_file)
            logg(f"common weights imported with acc {highest_acc}") 
            common_start_acc=highest_acc
            common_weights_loaded=True
            offspring_list=jax_create_offsprings((meta+numpy_seed),n_offsprings, father_weights,std_modifier)
            if use_father:
              offspring_list[0]=father_weights
            best_performer[0]=highest_acc

    start_meta = time.time()




    '''Offspring Decay'''
    if use_offspring_decay:
        off_base=sigma_decay(offspring_start, offspring_goal, n_decay_epochs)
        if meta < n_decay_epochs:
          n_offsprings=int(offspring_start*off_base**meta)
        else:
          n_offsprings=int(offspring_start*off_base**n_decay_epochs)

    '''Offspring Decay'''
    if use_sigma_decay:
        sigma_base=sigma_decay(sigma_start, sigma_goal, n_decay_epochs)
        if meta < n_decay_epochs:
          std_modifier=sigma_start*sigma_base**meta
        else:
          std_modifier=sigma_start*sigma_base**n_decay_epochs


    '''Load starting point'''
    if meta ==0 and not common_weights_loaded:
        
        if use_pickle:
            with open(pickle_path, "rb") as input_file:
              father_weights = cPickle.load(input_file)
              print("weights imported") 
            offspring_list=jax_create_offsprings((meta+numpy_seed),n_offsprings, father_weights,std_modifier)
            if use_father:
              offspring_list[0]=father_weights
        elif use_best_weights:
            father_weights = best_weights
            offspring_list=jax_create_offsprings((meta+numpy_seed),n_offsprings, father_weights,std_modifier)
            if use_father:
              offspring_list[0]=father_weights
        else:
          father_weights = conv_init(father_key, (batch_size,28,28,1))
          father_weights = father_weights[1] ## Weights are actually stored in second element of two value tuple
          offspring_list=jax_create_offsprings((meta+numpy_seed),n_offsprings, father_weights,std_modifier)
          if use_father:
            offspring_list[0]=father_weights

    '''Weight updates'''      
    if meta >=1:

        '''KNN weight update'''
        if use_KNN:
          KNN_used, KNN_offlist=KNN_weight_update(result_list_metaepoch, offspring_list)
          if KNN_used:
            offspring_list=[]
            offspring_list.append(best_weights)
            for off in KNN_offlist:
              offspring_list.append(off)
              offspring_list.extend(jax_create_offsprings((meta+numpy_seed),n_KNN_subsprings, father_weights,std_modifier))
          else:
              offspring_list=KNN_offlist

        '''Softmax Update'''
        if use_Softmax and not common_weights_loaded:
          grand_father=offspring_list[0]
          if not common_weights_loaded:
            father_weights=softmax_offlist(offspring_list,[x[0] for x in result_list_metaepoch],temp)
          offspring_list=jax_create_offsprings((meta+numpy_seed),n_offsprings, father_weights,std_modifier)
          if use_father:
            offspring_list[0]=grand_father
            offspring_list[1]=best_weights
            offspring_list[2]=father_weights
        
        '''Focus Weight Update'''
        if use_focus:
          grand_father=offspring_list[0]
          if meta % focus_change_every ==0:
            f_idx=f_idx+1
            logg(f"Focus change to layer {focus_layer[f_idx]}")
          offspring_list=create_focus_offsprings(n_offsprings, father_weights,std_modifier, focus_layer[f_idx])
          if use_father:
            offspring_list[0]=grand_father
            offspring_list[1]=best_weights
            offspring_list[2]=father_weights

          
    result_list_metaepoch=[]

    '''same data for every offspring'''
    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=n_offsp_epoch*n_samples,
                                                          test_size=n_offsp_epoch*n_test,stratify=y,
                                                          random_state=(starting_key+meta))
    
    x_train=x_train.reshape(n_offsp_epoch,int((n_samples/batch_size)),batch_size,28,28,1)
    y_train=y_train.reshape(n_offsp_epoch,int((n_samples/batch_size)),batch_size)
    x_test=x_test.reshape(n_offsp_epoch,n_test,28,28,1)
    y_test=y_test.reshape(n_offsp_epoch,n_test)
    
    #print("\tLänge Offspring List:",len(offspring_list))
    #print(f"\tTime overhead: {(time.time()-start_meta):.2f}s")

    for i in range(len(offspring_list)):

      conv_weights=offspring_list[i]
      result_off=jit_vmap_bootstrapp_offspring_MLP(rng_MLP,conv_weights,x_train,y_train,x_test,y_test)
      result_off2=[float(jnp.mean(result_off)),float(jnp.std(result_off))]
      result_list_metaepoch.append(result_off2)


      '''Check for best performer'''
      if result_off2[0]>best_performer[0]:
        
        best_performer=result_off2
        best_weights=conv_weights
        with open(save_path+f"best_weight_{result_off2[0]:.4f}.pkl", 'wb') as f:
          pickle.dump(best_weights, f, pickle.HIGHEST_PROTOCOL)
          f.close()
        logg(f"New best performer mean: {best_performer[0]:.4f}, std: {best_performer[1]:.2f}")
      
    logg("\tMetaepoch mean: {:.4f}, std: {:.2f}".format(np.mean(np.array([x[0] for x in result_list_metaepoch])),np.std(np.array([x[0] for x in result_list_metaepoch]))))
    logg("\tMetaepoch max performer: {:.4f}, min performer: {:.4f}".format(np.max(np.array([x[0] for x in result_list_metaepoch])),np.min(np.array([x[0] for x in result_list_metaepoch]))))
    logg("\tTime per metaepoch:{:.1f}s\n".format(time.time() - start_meta))
    results_meta.append(np.mean(np.array(result_list_metaepoch), axis=0))

Streaming output truncated to the last 5000 lines.

	Metaepoch mean: 0.8781, std: 0.00
	Metaepoch max performer: 0.8781, min performer: 0.8762
	Time per metaepoch:11.2s

	Metaepoch mean: 0.8769, std: 0.00
	Metaepoch max performer: 0.8775, min performer: 0.8763
	Time per metaepoch:11.2s

	Metaepoch mean: 0.8799, std: 0.00
	Metaepoch max performer: 0.8799, min performer: 0.8787
	Time per metaepoch:11.2s

	Metaepoch mean: 0.8739, std: 0.00
	Metaepoch max performer: 0.8743, min performer: 0.8731
	Time per metaepoch:11.3s

	Metaepoch mean: 0.8752, std: 0.00
	Metaepoch max performer: 0.8755, min performer: 0.8741
	Time per metaepoch:11.2s

	Metaepoch mean: 0.8703, std: 0.00
	Metaepoch max performer: 0.8704, min performer: 0.8687
	Time per metaepoch:11.2s

	Metaepoch mean: 0.8752, std: 0.00
	Metaepoch max performer: 0.8755, min performer: 0.8744
	Time per metaepoch:11.2s

	Metaepoch mean: 0.8732, std: 0.00
	Metaepoch max performer: 0.8734, min performer: 0.8712
	Time per metaepoch:11.3s

	Met

In [ ]:
weights

'JAX_MNist.ipynb'

# **Testing**

In [ ]:

with open("/content/drive/MyDrive/Colab Notebooks/MNist/best_weight.pkl", 'wb') as f:

        pickle.dump(best_weights, f, pickle.HIGHEST_PROTOCOL)
        f.close()

In [ ]:

dicta = [()] * len(father_weights)
for idx,w in enumerate(father_weights):
    if w:
        w, b = w
        #print("Weights : {}, Biases : {}".format(w.shape, b.shape))
  
        '''if weight layer only contains 0 and 1, only copy original weight layer, dont add random noise. Purpose of these 0 and 1 layers unclear'''
        if any(w[0].shape==t for t in [(Convu1_in,) ,(Convu2_in,), (Convu3_in,)]):
          x_w=w
          x_b=b
        else:
          seed=np.random.randint(0,100000)
          key = random.PRNGKey(seed)
          x_w = w+random.normal(key,shape=w.shape)*std_modifier
          x_b = b+random.normal(key,shape=b.shape)*std_modifier
          print("\n\n\n")
          print("diff x_w",jax.tree_map(lambda x,y: x-y, x_w,w))
          print("\n\n\n")
          print("diff x_b",jax.tree_map(lambda x,y: x-y, x_b,b))
          print("\n\n\n")
        dicta[idx]=(x_w,x_b)


In [ ]:
from collections import Counter

words = list(np.array(y_test))

print(Counter(words).keys()) # equals to list(set(words))
print(Counter(words).values()) # counts the elements' frequency


In [ ]:
  import os
  path=os.path.join("/content/long_run_fatherweights_04.04.22.pkl")
  with open(path, 'wb') as f:

    pickle.dump(father_weights, f, pickle.HIGHEST_PROTOCOL)
    f.close()

# Archiv

In [ ]:
def accuracy_test(test_images,test_lbls):
  acc_test_images=test_images.reshape(-1,28,28)
  acc_test_images=acc_test_images[:, :,:,np.newaxis]
  #print(acc_test_images.shape)

  acc_test_images = conv_apply(weights, acc_test_images)
  #print(imgs.shape)
  acc_test_images = acc_test_images.reshape(*acc_test_images.shape[:1],-1) # Flatten


  #print("test_lbls",test_lbls.shape)
  #print("test_images",acc_test_images.shape)
  return accuracy(MLP_params, acc_test_images, test_lbls)

In [ ]:
#funkioniert nicht
n_offspr=100
fath_weights=father_weights

statedic_list = jnp.empty(len(father_weights))
#statedic_list=[]
for i in range(0,n_offspr):
  dicta = jnp.empty(0) 
  #dicta = [()] * len(father_weights)
  for idx,w in enumerate(father_weights):
      if w:
          w, b = w
          #print("Weights : {}, Biases : {}".format(w.shape, b.shape))
    
          '''if weight layer only contains 0 and 1, only copy original weight layer, dont add random noise. Purpose of these 0 and 1 layers unclear'''
          if any(w[0].shape==t for t in [(Convu1_in,) ,(Convu2_in,), (Convu3_in,)]):
            x_w=w
            x_b=b
          else:
            #seed=random.randint(rng,(0,100000)
            key = random.PRNGKey(0)
            x_w = w+random.normal(key,shape=w.shape)*std_modifier
            x_b = b+random.normal(key,shape=b.shape)*std_modifier
            weight=jnp.empty(0) 
            weight=jnp.append(x_w,x_b, axis=None)
          dicta=jnp.append(dicta,weight, axis=None)
          print(jnp.shape(dicta))
  statedic_list=jnp.append(statedic_list, dicta, axis=None)
affe=statedic_list

In [ ]:
conv_weights=jax.tree_map(lambda x: x+jax.random.normal(convu_key)*std_modifier, father_weights)

diff=jax.tree_map(lambda x,y: x-y, father_weights,conv_weights)

jax.flatten_util.ravel_pytree(conv_weights)

In [ ]:
%%time
'''VMap/Batch of whole metaepoch. Runnning with 5 metaepochs, Kernel crushing with 10'''
#vmap_offspring_run



father_weights = conv_init(rng, (batch_size,28,28,1))
father_weights = father_weights[1] ## Weights are actually stored in second element of two value tuple


x_train=train_images[random.randint(rng, (n_metaepochs,n_offsp_epoch*n_samples), 0, 60000, dtype='uint8')]
y_train=train_lbls[random.randint(rng, (n_metaepochs,n_offsp_epoch*n_samples), 0, 60000, dtype='uint8')]
x_train=x_train.reshape(n_metaepochs,n_offsp_epoch,int((n_samples/batch_size)),batch_size,28,28,1)
y_train=y_train.reshape(n_metaepochs,n_offsp_epoch,int((n_samples/batch_size)),batch_size)

test_img_off=test_images[random.randint(rng, (n_metaepochs,n_offsp_epoch*n_test,), 0, 10000, dtype='uint8')] #n_testing_epochs not implemented, only running one testing epoch per offspring epoch
test_lbl_off=test_lbls[random.randint(rng, (n_metaepochs,n_offsp_epoch*n_test,), 0, 10000, dtype='uint8')]
test_img_off=test_img_off.reshape(n_metaepochs,n_offsp_epoch,n_test,28,28,1)
test_lbl_off=test_lbl_off.reshape(n_metaepochs,n_offsp_epoch,n_test)

print(jnp.shape(x_train))
print(jnp.shape(y_train))
print(jnp.shape(test_img_off))
print(jnp.shape(test_lbl_off))
key_matrix_seed=jnp.full((n_metaepochs,),np.arange(0, n_metaepochs, 1))

affe=vmap_offspring_run(key_matrix_seed,x_train,y_train,test_img_off,test_lbl_off,rngkey_MLP)
print(affe)

In [ ]:
'''VMap/Batch of whole metaepoch. Runnning with 5 metaepochs, Kernel crushing with 10'''
def vmap_offspring_run(key_matrix_seed,x_train,y_train,test_img_off,test_lbl_off,rngkey_MLP):
  return vmap(offspring_run, ( 0,0, 0,0,0,None))(key_matrix_seed,x_train,y_train,test_img_off,test_lbl_off,rngkey_MLP)

@jit
def offspring_run(key_matrix_seed,x_train,y_train,test_img_off,test_lbl_off,rngkey_MLP):

  
  
  rng=jax.random.PRNGKey(key_matrix_seed)
  key_matrix = conv_init(rng, (batch_size,28,28,1))[1]
  key_matrix=jax.tree_map(lambda x: x*std_modifier, key_matrix)
  conv_weights=jax.tree_map(lambda x,y: x+y, father_weights,key_matrix)

  
  result_off=jit_vmap_bootstrapp_offspring_MLP(rngkey_MLP,conv_weights,x_train,y_train,test_img_off,test_lbl_off)
  return(jnp.mean(result_off))
  #jnp.std(result_off)


In [ ]:

train_loader = torch.utils.data.DataLoader(torchvision.datasets.MNIST('/files/', train=True, download=True,
                                            transform=torchvision.transforms.Compose([
                                            torchvision.transforms.ToTensor(),
                                            torchvision.transforms.Normalize((0.1307,), (0.3081,))])),
                                            batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn,drop_last=True)

test_loader = torch.utils.data.DataLoader(torchvision.datasets.MNIST('/files/', train=False, download=True,
                                          transform=torchvision.transforms.Compose([
                                          torchvision.transforms.ToTensor(),
                                          torchvision.transforms.Normalize((0.1307,), (0.3081,))])),
                                          batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn,drop_last=True)

In [ ]:
def save_p(obj, name,direc):
    path=os.path.join(direc,name)
    with open(path+".pkl", 'wb') as f:

        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        f.close()


    

  
def pathandstuff():

    global save_txt
    global base_path
    global save_path

    if os.path.exists(local_path):
        '''Save running code file to log folder'''
        #nb_full_path = os.path.join(os.getcwd(), nb_name) #path of current notebook
        #shutil.copy2(nb_full_path, save_path) #save running code file to log folder
        print("on local")
        base_path=local_path
    elif os.path.exists(googledrive_path):
        print("on google")
        base_path=googledrive_path
    else:
        raise ValueError('Please specify save path or connect to Google Drive')
        
    logs_path=base_path+"Logs/"
    '''Set logging and temp paths'''
    timestamp=time.strftime("%d.%m.%Y_%H.%M")
    foldername=timestamp
    save_path=os.path.join(logs_path,foldername)
    print("Log path:",save_path)
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    save_txt = os.path.join(save_path, 'Log_MNist_{}.txt'.format(foldername))
    
   
def plot_mean(mean_results,std_results): 
    mean=mean_results
    std=std_results
    x_axis=np.arange(0,len(mean), 1)

    plt.xticks(range(0,len(mean)))

    mean = np.array(mean)
    std = np.array(std)

    lower_bound = mean - std
    upper_bound = mean + std

    plt.fill_between(x_axis, lower_bound, upper_bound, alpha=.3)
    plt.plot(mean,marker="o")
    plt.savefig(save_path+'/plot.png')
    return plt    

def sigma_decay(start, end, n_iter):
  return(end/start)**(1/n_iter)

def index_nhighest_list(n,list):
  return [i for x, i in heapq.nlargest(n,((x, i) for i, x in enumerate(list)))]

def matrix_distance(offspring_list):
    length=len(offspring_list)

    matrix=np.zeros((length, length))
    for c in range(length):
        for r in range(length):



            hilf=substract_two_dics(offspring_list[c],offspring_list[r])

            hilf2=square_dic(dict(hilf))
            sum_=dic_totalsum(hilf2)
            matrix[r][c]=sum_
    return matrix
    
def substract_two_dics(a,b):
    a = Counter(a)
    a.subtract(b)
    return a

'''logging to txt and print'''
def logg_to_file (string_, array=None):
  if array is None:

    file1 = open(save_txt,"a")
    file1.write(string_)
    file1.write("\n")
    file1.close()
    
  if array is not None:

    file1 = open(save_txt,"a")
    file1.write(string_)
    file1.write(str(array))
    file1.write("\n")
    file1.close()
    
def log_variables():
    logg_to_file(("seed = {}".format(seed)))
    logg_to_file (("n_training_epochs = {}".format(n_training_epochs)))
    logg_to_file (("n_offsp_epoch = {}".format(n_offsp_epoch)))
    logg_to_file (("n_testing_epochs = {}".format(n_testing_epochs)))
    logg_to_file (("n_samples = {}".format(n_samples)))
    logg_to_file (("n_test = {}".format(n_test)))
    logg_to_file (("batch_size_train = {}".format(batch_size_train)))
    logg_to_file (("learning_rate = {}".format(learning_rate)))

    logg_to_file (("std_modifier = {}".format(std_modifier)))
    logg_to_file (("use_sigma_decay = {}".format(use_sigma_decay)))
    logg_to_file (("sigma_start = {}".format(sigma_start)))
    logg_to_file (("sigma_goal = {}".format(sigma_goal)))
    logg_to_file (("elitist = {}".format(elitist)))
    logg_to_file (("NNin1 = {}".format(NNin1)))
    logg_to_file (("NNout1 = {}".format(NNout1)))
    logg_to_file (("Convu_in1 = {}".format(Convu_in1)))
    logg_to_file (("Convu_out1 = {}".format(Convu_out1)))
    logg_to_file (("Convu_out2 = {}".format(Convu_out2)))

    logg_to_file (("kernelsize_ = {}".format(kernelsize_)))
    logg_to_file (("use_elitist = {}".format(use_elitist)))
    logg_to_file (("n_metaepochs = {}".format(n_metaepochs)))
    logg_to_file (("n_testing_epochs = {}".format(n_testing_epochs)))
                  
    logg_to_file (("n_offsp_epoch = {}".format(n_offsp_epoch)))
    logg_to_file (("n_offsprings = {}".format(n_offsprings)))

    logg_to_file (("use_softmax = {}".format(use_softmax)))
    logg_to_file (("temperature = {}".format(temperature)))

                  
def square_dic(my_dict):
    my_dict.update((x, y**2) for x, y in my_dict.items())
    return my_dict   

def dic_totalsum(my_dict):
    return float(torch.sum(sum(sum(x) for x in my_dict.values())))

'''weighting every element in dic list with softmax weights from m'''
def weight_offspring_list(n_offsprings,offspring_list,m):
    for n in range(n_offsprings):

        for key in offspring_list[n]:    
            offspring_list[n][key] *=  m[n]
    return offspring_list

'''needed for softmax'''
def sum_list_of_dics(dic_list):
    c = Counter()
    for d in w_offspring_list:
        c.update(d)
    return c

'''get best performer from final resultlist[[0,2,0.8],[1,2,0.5],[2,2,0.2]'''
def best_index_bestperformer(lista):
    hilf=0
    index=None
    for i in range (len(lista)):
        if hilf<lista[i][2].item():
            index=i
            hilf=lista[i][2].item()
    return index



In [ ]:
'''softmax from keylist for approach 1'''
def softmax_conv_weights(father_weights,keylist, acc_list,temp):
  temp_list=softmax(acc_list,temp)
  for k in range(len(keylist)):
        if k==0:
          top_dog = father_weights
          top_dog=jax.tree_map(lambda x: x*temp_list[k], top_dog)
        else:
          rng_soft = jax.random.PRNGKey(k)
          general_dog = conv_init(rng_soft, (batch_size,28,28,1))[1]
          top_dog=jax.tree_map(lambda x,y: x+y*std_modifier*temp_list[k], top_dog,general_dog)
  return top_dog

In [ ]:
'''#test better performer 1
conv_init, conv_apply = stax.serial(
    stax.Conv(Convu1_in,(3,3), strides=(1,1),padding="SAME"),
    stax.BatchNorm(),
    stax.Relu,
    stax.MaxPool((2,2)),
    stax.Conv(Convu2_in, (3,3), strides=(1,1),padding="SAME"),
    stax.BatchNorm(),
    stax.Relu,
    stax.MaxPool((2,2)),
    
)
'''

In [ ]:
'''Convu1_in=12
Convu2_in=24
#test better performer 2, good performance
conv_init, conv_apply = stax.serial(
    stax.Conv(Convu1_in,(3,3), strides=(1,1),padding="SAME"),
    stax.BatchNorm(),
    stax.Relu,
    stax.MaxPool((2,2)),
    stax.Conv(Convu2_in, (3,3), strides=(1,1),padding="SAME"),
    stax.BatchNorm(),
    stax.Relu,
    stax.MaxPool((2,2)),
   
)
'''

In [ ]:
''' #Source Robert Lange
conv_init, conv_apply = stax.serial(stax.Conv(12, (5, 5), (1, 1), padding="SAME"),
                                 stax.BatchNorm(), stax.Relu,
                                 stax.Conv(24, (5, 5), (1, 1), padding="SAME"),
                                 stax.BatchNorm(), stax.Relu,
                                 stax.Conv(24, (3, 3), (1, 1), padding="SAME"),
                                 stax.BatchNorm(), stax.Relu,
                                 stax.Conv(10, (3, 3), (1, 1), padding="SAME"), stax.BatchNorm(),stax.Relu
                                 )'''

In [ ]:
'''train_dataset = MNIST(root='train_mnist', train=True, download=True,transform=torchvision.transforms.Compose([
                                            torchvision.transforms.ToTensor(),
                                            torchvision.transforms.Normalize((0.1307,), (0.3081,))]))

test_dataset = MNIST(root='test_mnist', train=False, download=True,transform=torchvision.transforms.Compose([
                                            torchvision.transforms.ToTensor(),
                                            torchvision.transforms.Normalize((0.1307,), (0.3081,))]))

train_images = jnp.array(train_dataset.data,dtype="float32").reshape(len(train_dataset), -1)
train_lbls = jnp.array(train_dataset.targets)

test_images = jnp.array(test_dataset.data,dtype="float32").reshape(len(test_dataset), -1)
test_lbls = jnp.array(test_dataset.targets)
'''

In [ ]:
file = open("/content/long_run_fatherweights_04.04.22.pkl",'rb')
object_file = pickle.load(file)
file.close()

with open(r"/content/long_run_fatherweights_04.04.22.pkl", "rb") as input_file:
  father_weights = cPickle.load(input_file)

In [ ]:
def log_variables():
    
    logg_to_file (("n_training_epochs = {}".format(n_training_epochs)))
    logg_to_file (("n_offsp_epoch = {}".format(n_offsp_epoch)))
    
    logg_to_file (("n_samples = {}".format(n_samples)))
    logg_to_file (("n_test = {}".format(n_test)))
    logg_to_file (("batch_size = {}".format(batch_size)))
    

    logg_to_file (("std_modifier = {}".format(std_modifier)))
    logg_to_file (("use_sigma_decay = {}".format(use_sigma_decay)))
    logg_to_file (("sigma_start = {}".format(sigma_start)))
    logg_to_file (("sigma_goal = {}".format(sigma_goal)))
    logg_to_file (("n_decay_epochs = {}".format(n_decay_epochs)))
    logg_to_file (("use_pickle = {}".format(use_pickle)))
    logg_to_file (("use_father = {}".format(use_father)))

    logg_to_file (("NNin1 = {}".format(NNin1)))
    logg_to_file (("NNout1 = {}".format(NNout1)))
    logg_to_file (("Convu_in1 = {}".format(Convu_in1)))
    logg_to_file (("Convu_out1 = {}".format(Convu_out1)))
    logg_to_file (("Convu_out2 = {}".format(Convu_out2)))

    logg_to_file (("kernelsize_ = {}".format(kernelsize_)))
    
    logg_to_file (("n_metaepochs = {}".format(n_metaepochs)))
    logg_to_file (("n_testing_epochs = {}".format(n_testing_epochs)))
                  
    logg_to_file (("n_offsp_epoch = {}".format(n_offsp_epoch)))
    logg_to_file (("n_offsprings = {}".format(n_offsprings)))

    logg_to_file (("use_softmax = {}".format(use_softmax)))
    logg_to_file (("temperature = {}".format(temperature)))